In [10]:
import mysql.connector
from mouseBehavior import mouseBehaviorData as mbd
from matplotlib import pyplot as plt
import os
import pandas as pd
import numpy as np

In [11]:
mouseID = '491252'

In [12]:
def checkforpickle(directory, mouseID):
    pkllist = [f for f in os.listdir(directory) if '.pkl' in f]
    mouseID = str(mouseID)
    pklpath = None
    for pkl in pkllist:
        if pkl[:6]==mouseID:
            pklpath = os.path.join(directory, pkl)
    return pklpath

In [13]:
pkldir = r"\\allen\programs\braintv\workgroups\nc-ophys\corbettb\NP_behavior_pipeline\behavior_sessions\Ramirez\behavior_sessions"
dataframe_save_dir = r"\\allen\programs\braintv\workgroups\nc-ophys\corbettb\NP_behavior_pipeline\behavior_sessions\Ramirez\behavior_sessions\dataframes"

In [ ]:
saved_df_dir = r"\\allen\programs\braintv\workgroups\nc-ophys\corbettb\NP_behavior_pipeline\behavior_sessions"
df_paths = [os.path.join(saved_df_dir, f) for f in os.listdir(saved_df_dir)]

mouse_dict = {a:{} for a in [f.split('\\')[-1][:6] for f in df_paths]}
for df_path in df_paths:
    m = mbd()
    m.loadFromPickle(df_path)
    m.buildBehaviorDataframe(all_sessions=True)
    
    mouse_dict[m.mouse_id]['df_obj'] = m

In [14]:
#First look to see if there's already a pkl file with this mouse' data
pklpath = checkforpickle(pkldir, mouseID)

if pklpath is not None:
    print('reading from pickle')
    m = mbd()
    m.loadFromPickle(pklpath)
    m.add_weight_and_water_history()
    m.calculate_behavior_metrics()
    m.add_metadata_to_dataframe()
    m.beh_df.drop(columns=['cumulative_rewards'], inplace=True)
    m.save_dataframe_separately(dataframe_save_dir)

else:
    print('fetching data for '+mouseID)
    m = mbd(mouse_id=mouseID, daysBeforeHandoff=100)
    m.buildBehaviorDataframe(all_sessions=True)
print('done')
# m.saveDirectory = pkldir
# m.saveToPickle()

fetching data


ImportError: No module named pymssql

In [ ]:
fig, ax = plt.subplots()
ax.plot(m.beh_df['session_datetime_local'], m.beh_df['Wt_g'], 'ko')
ax.set_title('Wt_g')

fig, ax = plt.subplots()
ax.plot(m.beh_df['session_datetime_local'], m.beh_df['WE_ml'], 'bo')
ax.plot(m.beh_df['session_datetime_local'], m.beh_df['WS_ml'], 'ro')
ax.legend(['WE_ml', 'WS_ml'])

fig, ax = plt.subplots()
ax.plot(m.beh_df['session_datetime_local'], np.array(m.beh_df['Wt_g'].astype(float) - m.beh_df['WE_ml'].astype(float)), 'go')
ax.set_title('Wt_g - WE_ml: inferred pre-session weight')


In [ ]:
m.beh_df[['session_datetime_local', 'Wt_g', 'stage']]

In [ ]:
m.beh_df.head()